# Анализ базы данных сервиса для чтения книг по подписке с помощью SQL

Цель: анализ базы данных.

Задачи:
- посчитать, сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитать количество обзоров и среднюю оценку;
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': , # имя пользователя
 'pwd': , # пароль
 'host': ,
 'port': , # порт подключения
 'db': } # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
books_query = '''SELECT * FROM books'''
authors_query = '''SELECT * FROM authors'''
publishers_query = '''SELECT * FROM publishers'''
ratings_query = '''SELECT * FROM ratings'''
reviews_query = '''SELECT * FROM reviews'''

In [3]:
for i in [books_query, authors_query, publishers_query, ratings_query, reviews_query]:
    display(pd.io.sql.read_sql(i, con = engine).head(1))
    display(pd.io.sql.read_sql(i, con = engine).info())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

,author_id,author
0,1,A.S. Byatt


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

In [4]:
query_1 = '''SELECT COUNT(DISTINCT book_id)
             FROM books
             WHERE publication_date > '2000-01-01'
          '''

In [5]:
pd.io.sql.read_sql(query_1, con = engine) 

,count
0,819


После 1 января 2000 года вышло 819 книг.

In [6]:
query_2 = '''SELECT books.book_id, books.title,
             COUNT(DISTINCT reviews.review_id) AS reviews,
             AVG(ratings.rating) AS avg_rating
             FROM
             books
             LEFT JOIN reviews ON reviews.book_id = books.book_id
             LEFT JOIN ratings ON ratings.book_id = books.book_id
             GROUP BY books.book_id
             ORDER BY avg_rating DESC, reviews DESC
             '''

In [7]:
pd.io.sql.read_sql(query_2, con = engine)

,book_id,title,reviews,avg_rating
0,17,A Dirty Job (Grim Reaper #1),4,5.00
1,553,School's Out—Forever (Maximum Ride #2),3,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
4,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,202,Drowning Ruth,3,2.00
997,316,His Excellency: George Washington,2,2.00
998,371,Junky,2,2.00


Для каждой книги подсчитаны количество обзоров и средняя оценка.

In [8]:
query_3 = '''SELECT COUNT(books.book_id) AS count,
             publishers.publisher
             FROM
             books
             INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
             WHERE books.num_pages > 50
             GROUP BY books.publisher_id, publishers.publisher
             ORDER BY count DESC
             LIMIT 1
             '''

In [9]:
pd.io.sql.read_sql(query_3, con = engine)

,count,publisher
0,42,Penguin Books


Penguin Books - издательство, которое выпустило наибольшее число (42) книг толще 50 страниц.

In [10]:
query_4 = '''SELECT Sub.author, AVG(Sub.avg) AS avg_rating FROM
            (SELECT authors.author AS author, books.book_id, AVG(ratings.rating) AS avg
             FROM
             authors
             INNER JOIN books ON books.author_id = authors.author_id
             INNER JOIN ratings ON ratings.book_id = books.book_id
             GROUP BY ratings.book_id, books.book_id, authors.author
             HAVING COUNT(ratings.rating) > 50
             ORDER BY authors.author) AS Sub
             GROUP BY Sub.author
             ORDER BY avg_rating DESC
             
             '''

In [11]:
pd.io.sql.read_sql(query_4, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

In [12]:
query_5 = '''SELECT AVG(Sub.count)
             FROM
             (SELECT username,  COUNT(review_id) AS count
             FROM
             reviews
             WHERE username IN 
             (SELECT username
             FROM
             ratings
             GROUP BY username
             HAVING COUNT(rating) > 50)
             GROUP BY username) AS Sub
            
             '''

In [13]:
pd.io.sql.read_sql(query_5, con = engine)

,avg
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, - 24.

## Вывод

После 1 января 2000 года вышло 819 книг.

Penguin Books - издательство, которое выпустило наибольшее число (42) книг толще 50 страниц.

Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré.

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, - 24.